# Simple Application

This notebook is created as an application for using the trained model on new audio and give a classification based on the 41 different classes:

 > 'Hi-hat' 'Saxophone' 'Trumpet' 'Glockenspiel' 'Cello' 'Knock'
 'Gunshot_or_gunfire' 'Clarinet' 'Computer_keyboard' 'Keys_jangling'
 'Snare_drum' 'Writing' 'Laughter' 'Tearing' 'Fart' 'Oboe' 'Flute' 'Cough'
 'Telephone' 'Bark' 'Chime' 'Bass_drum' 'Bus' 'Squeak' 'Scissors'
 'Harmonica' 'Gong' 'Microwave_oven' 'Burping_or_eructation' 'Double_bass'
 'Shatter' 'Fireworks' 'Tambourine' 'Cowbell' 'Electric_piano' 'Meow'
 'Drawer_open_or_close' 'Applause' 'Acoustic_guitar' 'Violin_or_fiddle'
 'Finger_snapping'
 
The application is built using ipywidgets and voilà. Ipywidgets makes it easy to make buttons and display the audio and results. Voilà removes everything but the widgets so that it looks clean and all code is hidden.

# Imports & setup

In [38]:
import librosa
from IPython.display import Audio
import pathlib
from pathlib import Path
from fastai.vision.all import *
from fastai.vision.widgets import *

In [46]:
# Supress warnings that appear when uploading and classifying audio.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load model

The path is just set to be the path used in the github repo. Change it to where you have the model located.
```
├───models
│   └─── model_V2.pkl
└───nbs
    ├─── Application.ipynb
    └───.ipynb_checkpoints
```

In [40]:
path = Path('../')
model_path = path/'models'

# This try/catch fixes an error on windows when using PosixPath
try:
    learn_inf = load_learner(model_path/'model_V2.pkl')
except:
    posix_backup = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    learn_inf = load_learner(model_path/'model_V2.pkl')
    pathlib.PosixPath = posix_backup

# Creating widgets

In [41]:
# Creating the the widgets for uploading audio, 
# playing audio and displaying the prediction.
btn_upload = widgets.FileUpload(accept=".wav,.mp3")
out_pl = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred.value = 'Please select audio'

# Converting audio to images

This is the same function for converting audio to mel spectrograms as used in training the model, except for a few tweaks to make it work with the audio file from the uploader.

In [42]:
def log_mel_spec_tfm(audio_upload, dst_path = path/'../data/imgs/uploaded'):
    data, sample_rate = librosa.load(io.BytesIO(audio_upload.data[-1]))
    
    n_fft = 1024
    hop_length = 512
    n_mels = 80
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(data, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0, 
                                                    fmin=fmin, fmax=fmax)
    
    #mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate)
    
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
    
    
    dst_path.mkdir(exist_ok=True)
    fname = list(audio_upload.value)[0]
    
    plt.imsave(dst_path / (fname[:-4] + '.png'), mel_spec_db)
    
    return dst_path / (fname[:-4] + '.png')

# Use the model on uploaded data

When an audiofile is uploaded this function will run which makes a prediction on the image representation of the audio.

In [43]:
# Change dst_path to where you want the spectrogram 
# of your uploaded audio to be saved.
def on_click_classify(change):
    dst_path = path/'../data/imgs/uploaded'
    out_pl.clear_output()
    with out_pl: display(Audio(btn_upload.data[-1]))
    audio_img = log_mel_spec_tfm(btn_upload, dst_path)
    pred,pred_idx,probs = learn_inf.predict(audio_img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [44]:
btn_upload.observe(on_click_classify, names=['data'])

The VBox gathers all the widgets specified and is what makes it look nice and tidy.

If you upload multiple audio files, it will only use the last file uploaded to create a prediction.

In [45]:
widgets.VBox([widgets.Label('Select your audio'),
      btn_upload, out_pl, lbl_pred])

In [28]:
#!pip install voila
#!jupyter serverextension enable voila --sys-prefix